## Contents
- [生成器](#generator)
    - [send()](#send)
    - [throw()](#throw)
    - [生成器应用](#apply)
        - [斐波那契数列](#fibonacci)
        - [动态统计平均数](#running_average)

## <a id='generator'>生成器</a>

### <a id='send'>send()方法</a>

紧接着已执行完的yield语句后发送，并执行下一个yield语句

In [12]:
def gen():
    while True:
        x = yield 'yes'
        print('received:', x)

In [13]:
g = gen()

In [14]:
next(g)  #需要先启动生成器才能send

'yes'

In [15]:
next(g)  #next语句实际上分2步，先send(None), 再执行到下一个yield语句

received: None


'yes'

In [16]:
g.send(89)  #紧接着已执行完的yield语句后发送，并执行到下一个yield语句

received: 89


'yes'

In [20]:
def init_genetor(generator):
    "修饰生成器函数，返回一个已经启动的生成器"
    def wrapper(*args, **kwargs):
        g = generator(*args, **kwargs)
        next(g)
        return g
    return wrapper

In [21]:
@init_genetor
def generator():
    while True:
        x = yield
        if x:
            print('received:',x)
        else:
            print('received nothing')

In [22]:
g = generator()

In [23]:
type(g)

generator

In [24]:
g.send(120)

received: 120


### <a id='throw'>throw()方法</a>

In [10]:
@prepare
def gen():
    count = 0
    while True:  
        count += 1
        print('counting...') 
         
        try:
            x = yield 'yes'
        except Exception:
            print('yielded:', count)
            
        print('received:', x)
        

In [11]:
g = gen()

wrapper: <class 'generator'>
counting...


In [12]:
g.send(10)   #紧接着上一个被执行的yield之后，并执行下一个yield语句

received: 10
counting...


'yes'

In [13]:
g.throw(Exception)  #紧接着上一次执行的yield之后，并执行下一个yield

yielded: 2
received: 10
counting...


'yes'

In [14]:
next(g)       #紧接着上一个被执行的yield之后，并执行下一个yield语句

received: None
counting...


'yes'

## <a id='apply'>生成器应用</a>

### <a id='fibonacci'>斐波那契数列</a>

通过装饰器限制产出的斐波那契数列个数

弊端：无法灵活地用使用者控制输出序列个数

In [15]:
def count(n):
    def decorator(gen):
        def wrapper():
            g = gen()
            for i in range(n):
                print('counting:',i+1)
                yield next(g)
        return wrapper
    return decorator

@count(5)  
def fibonacci():
    a,b = 0,1
    while True:
        yield a
        a, b = b, a+b
        

In [16]:
list(fibonacci())  #list实际接收的只有yield语句的返回值

counting: 1
counting: 2
counting: 3
counting: 4
counting: 5


[0, 1, 1, 2, 3]

In [17]:
f = fibonacci()
for i in range(10):   #count()装饰器对原生成器进行了限制，原生成器是无限的，限制后的生成器只有5个
    print(next(f))    

counting: 1
0
counting: 2
1
counting: 3
1
counting: 4
2
counting: 5
3


StopIteration: 

#### 改进一：
改造生成器本身，限制序列个数

弊端：破坏了生成器的定义

In [18]:
def fibonacci_with_limit(n):
    a, b = 0, 1
    for i in range(n):
        yield a
        a, b = b, a+b

In [19]:
list(fibonacci_with_limit(10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

#### 改进二：
在生成器外面控制序列个数

In [20]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a+b

def limit(generator, n):
    for i in range(n):
        yield next(generator)

In [21]:
list(limit(fibonacci(), 10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

### <a id='running_average'>动态统计平均数</a>
Write a generator which computes the running average.     
通过send方法实时传数，动态计算平均值    

In [28]:
@init_genetor
def avg():
    sum = 0
    count = 0
    while True:
        avg = (sum/count) if count else None
        num = yield avg
        if num:
            count += 1
            sum += num
    

In [29]:
g = avg()

In [30]:
g.send(12)

12.0

In [31]:
g.send(18)

15.0

In [32]:
g.send(20)

16.666666666666668

In [33]:
g.send(100)

37.5

[官网写法](https://www.python-course.eu/python3_generators.php) :

In [34]:
def running_average():
    total = 0
    counter = 0
    average = 0.0
    while True:
        term = yield average
        counter += 1
        total += term
        average = total / counter

In [35]:
ra = running_average()

In [36]:
next(ra)  #初始化生成器

0.0

In [37]:
import random

In [40]:
for value in [random.randint(10,100) for i in range(10)]:
    out_str = "sent:{val:3d}, average:{avg:6.2f}"
    print(out_str.format(val=value, avg=ra.send(value)))

sent: 25, average: 25.00
sent: 25, average: 25.00
sent: 89, average: 46.33
sent: 66, average: 51.25
sent: 58, average: 52.60
sent: 62, average: 54.17
sent: 65, average: 55.71
sent: 15, average: 50.62
sent: 96, average: 55.67
sent: 64, average: 56.50
